In [ ]:
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv("used_cars.csv")

In [ ]:
#Show the first 5 data rows
data.head()

In [ ]:
#Convert mixed columns to numeric by removing non-numeric characters
data['Mileage'] = pd.to_numeric(data['Mileage'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
data['Engine'] = pd.to_numeric(data['Engine'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
data['Power'] = pd.to_numeric(data['Power'].str.extract('(\d+\.?\d*)')[0], errors='coerce')

#Remove columns that won't faciltate for prediction (e.g., 'S.No.', 'Name', and 'New_Price')
data = data.drop(columns=['S.No.', 'Name', 'New_Price'])

#Remove rows with missing values in the target variable 'Price' and do imputation for missing values in other columns
data = data.dropna(subset = ['Price'])
data.fillna(data.median(numeric_only = True), inplace = True)

#Use one-hot encoding to convert categorical columns
categorical_columns = ['Location', 'Fuel_Type', 'Transmission', 'Owner_Type']
data = pd.get_dummies(data, columns = categorical_columns, drop_first = True)

In [ ]:
#New data
data.head()

In [ ]:
#Correlation matrix
plt.figure(figsize=(14, 10))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1, square=True)
plt.title("Correlation Matrix for Loan Prediction Dataset")
plt.show()

#Identifying features with strong correlation to 'Price'
correlation_with_target = correlation_matrix['Price'].abs().sort_values(ascending=False)
correlation_with_target

In [ ]:
#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Applying Lasso Feature Selection
lasso = LassoCV(cv = 5, random_state = 42).fit(X_train, y_train)
lasso_selected_features = X.columns[(lasso.coef_ != 0)]

# Applying Recursive Feature Elimination (RFE)
linear_regressor = LinearRegression()
rfe = RFECV(estimator=linear_regressor, step = 1, cv = 5)
rfe.fit(X_train, y_train)
rfe_selected_features = X.columns[rfe.support_]

#Evaluating the model
y_pred = rfe.predict(X_test)
print("R^2:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))